# Una nueva normalidad 

<img src="images/highres.jpg" alt="170" width="850"/>

In [1]:
import numpy as np
import pandas as pd
from pymongo import MongoClient

In [2]:
# Two datasets are imported:
# prov reports on the situation of the 52 provinces
# excep handles the exceptions for all the municipalities inside the province

prov = pd.read_csv("data/Provincias.csv", dtype={"CP": object})
excep = pd.read_csv("data/Excepciones.csv", dtype={"CP": object, "Provincia": object})

In [3]:
display(prov.head())
display(excep.head())

,CP,Provincia,Fase,Excepcion
0,04,Almería,1,FALSE
1,11,Cádiz,1,FALSE
2,14,Córdoba,1,FALSE
3,21,Huelva,1,FALSE
4,23,Jaén,1,FALSE


,Provincia,CP,Municipio,Fase,Territorio
0,12,"12570, 12579",Alcalà de Xivert,1,Vinaròs
1,12,12579,Alcossebre,1,Vinaròs
2,12,12579,Santa Magdalena de Pulpis,1,Vinaròs
3,12,"12530, 12580",Benicarló,1,Vinaròs
4,12,12598,Peñíscola,1,Vinaròs


## 🧹 Handling the "CP" column

In [4]:
# The dataframe index becomes the code of the province 
# with an exception

byCode = excep.set_index(["Provincia"])

byCode

,CP,Municipio,Fase,Territorio
Provincia,,,,
12,"12570, 12579",Alcalà de Xivert,1,Vinaròs
12,12579,Alcossebre,1,Vinaròs
12,12579,Santa Magdalena de Pulpis,1,Vinaròs
12,"12530, 12580",Benicarló,1,Vinaròs
12,12598,Peñíscola,1,Vinaròs
...,...,...,...,...
08,08593,Tagamanent,0,Barcelona
08,"08470, 08471",Vallgorguina,0,Barcelona
08,08188,Vallromanes,0,Barcelona


In [5]:
# Several municipalities have several Zip Codes
# The column is exploded and cleaned

def clean_CP(df):
    df["CP"] = df["CP"].apply(lambda X: X.split(","))
    df = df.explode("CP")
    df["CP"] = df["CP"].apply(lambda X: X.replace(" ", ""))
    df["CP"] = df["CP"].apply(lambda X: X.replace("\t", ""))
    return df

df = clean_CP(byCode)
df.head()

,CP,Municipio,Fase,Territorio
Provincia,,,,
12,12570,Alcalà de Xivert,1,Vinaròs
12,12579,Alcalà de Xivert,1,Vinaròs
12,12579,Alcossebre,1,Vinaròs
12,12579,Santa Magdalena de Pulpis,1,Vinaròs
12,12530,Benicarló,1,Vinaròs


## 🛠 Handling the exceptions into a dict

### I want my database to be a nested dictionary where in the case of Excepcion = True, an object with all the zip codes that have an exception is returned.

In [6]:
# A dictionary of dataframes is created
# Each dataframe will be grouped by each Zip Code

codigos = byCode.index.value_counts().index.to_list()

dfDict = {codigo : df.loc[codigo] for codigo in codigos}

print(dfDict.keys())

dict_keys(['08', '03', '46', '49', '09', '24', '12', '47', '37', '05', '34', '38', '42', '07', '40', '35'])


In [7]:
# Each df of dfDict is converted to a dictionary of dictionaries

dictDict = {}
for i in dfDict:
    dictDict[i] = dfDict[i].to_dict("record")

print(dictDict["08"][0])

{'CP': '08731', 'Municipio': 'Avinyonet del Penedès', 'Fase': 0, 'Territorio': 'Barcelona'}


In [8]:
# Añado aparte el CP 35 porque es una serie y arriba me da error
"""
dictDict["35"] = dfDict["35"].to_dict()
dictDict.keys()"""

'\ndictDict["35"] = dfDict["35"].to_dict()\ndictDict.keys()'

In [9]:
# Trato cada diccionario de dataframes para que me devuelva la arquitectura que quiero

def parse_excepcion(dfDict, code):
    dictio ={}
    dictio[code] = dfDict[code].to_dict("record")
    d_temp = {}
    for territory in dictio[code]:
        d_temp[territory['CP']] = {'Fase':territory['Fase'],'Territorio':territory['Territorio']}

    return d_temp

In [10]:
# The initial prov df is converted to dictionary so I can change the value when "Excepcion" = True

data_dict = prov.to_dict("records")

print(data_dict[0])
print(data_dict[-1])

{'CP': '04', 'Provincia': 'Almería', 'Fase': '1', 'Excepcion': 'FALSE'}
{'CP': '49', 'Provincia': 'Zamora', 'Fase': 'Parcial', 'Excepcion': 'TRUE'}


In [11]:
# data_dict when Excepcion = True is changed
# Now it is parsed as I want it

for value in data_dict:
    if value['Excepcion']:
        prefijo = value["CP"]
        excepcion = parse_excepcion(dfDict, prefijo)
        value['Excepcion'] = excepcion
        
print(data_dict[0])
print(data_dict[-1])

KeyError: '04'

In [12]:
data_dict

[{'CP': '04', 'Provincia': 'Almería', 'Fase': '1', 'Excepcion': 'FALSE'},
 {'CP': '11', 'Provincia': 'Cádiz', 'Fase': '1', 'Excepcion': 'FALSE'},
 {'CP': '14', 'Provincia': 'Córdoba', 'Fase': '1', 'Excepcion': 'FALSE'},
 {'CP': '21', 'Provincia': 'Huelva', 'Fase': '1', 'Excepcion': 'FALSE'},
 {'CP': '23', 'Provincia': 'Jaén', 'Fase': '1', 'Excepcion': 'FALSE'},
 {'CP': '41', 'Provincia': 'Sevilla', 'Fase': '1', 'Excepcion': 'FALSE'},
 {'CP': '22', 'Provincia': 'Huesca', 'Fase': '1', 'Excepcion': 'FALSE'},
 {'CP': '44', 'Provincia': 'Teruel', 'Fase': '1', 'Excepcion': 'FALSE'},
 {'CP': '50', 'Provincia': 'Zaragoza', 'Fase': '1', 'Excepcion': 'FALSE'},
 {'CP': '33', 'Provincia': 'Asturias', 'Fase': '1', 'Excepcion': 'FALSE'},
 {'CP': '07', 'Provincia': 'Ballears', 'Fase': '1', 'Excepcion': 'FALSE'},
 {'CP': '38', 'Provincia': 'Tenerife', 'Fase': '1', 'Excepcion': 'FALSE'},
 {'CP': '35', 'Provincia': 'Las Palmas', 'Fase': '1', 'Excepcion': 'FALSE'},
 {'CP': '39', 'Provincia': 'Cantabria',

## 🌍 Connection to MongoDB server

In [ ]:
# Setting up the connection to the database

client = MongoClient()

def insertRecord(dictionary):
    db = client["unanuevanormalidad"] # Creo db
    collection = db["data"] # Creo collection
    collection.insert_many(dictionary)


In [ ]:
# Final dictionary is added to the database

#insertRecord(data_dict)

In [ ]:
data_dict[-1]